In [66]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('../../Data/Mendalay.csv')
df.head()

,Timestamp,PM2.5 [ug/m3],PM10[ug/m3],Ozone [ppb],Carbon_Monoxide [ppb],Temperature [°C],Relative_Humidity [%]
0,31/01/2022 12:00:00,13.181818,14.181818,456.090909,371.909091,20.413636,28.798182
1,31/01/2022 13:00:00,16.333333,17.333333,78.833333,405.750000,22.851667,22.753333
2,31/01/2022 14:00:00,25.480000,26.893333,44.280000,407.946667,24.910667,19.626133
3,31/01/2022 15:00:00,26.701058,28.343915,39.462963,405.023810,25.443915,19.002831
4,31/01/2022 16:00:00,32.166667,34.166667,33.916667,399.416667,23.152500,22.096667


In [8]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %H:%M:%S')
df.set_index('Timestamp', inplace=True)


In [51]:
def create_lagged_features(df, lags=3):
    for lag in range(1, lags+1):
        df[f"humidity_lag_{lag}"] = df['humidity'].shift(lag)
        df[f"temp_lag_{lag}"] = df['temperature'].shift(lag)
    df = df.dropna()  
    return df

In [52]:
df.head()

,PM2.5 [ug/m3],PM10[ug/m3],Ozone [ppb],Carbon_Monoxide [ppb],Temperature [°C],Relative_Humidity [%]
Timestamp,,,,,,
2022-01-31 12:00:00,13.181818,14.181818,456.090909,371.909091,20.413636,28.798182
2022-01-31 13:00:00,16.333333,17.333333,78.833333,405.750000,22.851667,22.753333
2022-01-31 14:00:00,25.480000,26.893333,44.280000,407.946667,24.910667,19.626133
2022-01-31 15:00:00,26.701058,28.343915,39.462963,405.023810,25.443915,19.002831
2022-01-31 16:00:00,32.166667,34.166667,33.916667,399.416667,23.152500,22.096667


In [6]:
columns_to_drop = ['PM2.5 [ug/m3]', 'PM10[ug/m3]', 'Ozone [ppb]', 'Carbon_Monoxide [ppb]']
df = df.drop(columns=columns_to_drop)

df = df.rename(columns={
    'Temperature [°C]': 'Temperature',
    'Relative_Humidity [%]': 'Humidity'
})

df.head()

,Timestamp,Temperature,Humidity
0,2022-01-31 12:00:00,20.413636,28.798182
1,2022-01-31 13:00:00,22.851667,22.753333
2,2022-01-31 14:00:00,24.910667,19.626133
3,2022-01-31 15:00:00,25.443915,19.002831
4,2022-01-31 16:00:00,23.152500,22.096667
